In [8]:
from dotenv import load_dotenv
from langchain.tools import tool
from langchain.agents import create_agent
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import InMemorySaver

load_dotenv()

True

In [9]:
# travel agent tool arg sanitizer for Kiwi MCP flight search tool
import re
from datetime import datetime
from typing import Any, Optional

from langchain_mcp_adapters.interceptors import MCPToolCallRequest


def _normalize_date_to_ddmmyyyy(value: Any) -> Optional[str]:
    if value is None:
        return None
    if not isinstance(value, str):
        return None
    s = value.strip()
    if not s:
        return None
    # Already in dd/mm/yyyy
    if re.fullmatch(r"\d{2}/\d{2}/\d{4}", s):
        return s
    # Try ISO formats
    for fmt in ("%Y-%m-%d", "%Y/%m/%d"):
        try:
            return datetime.strptime(s, fmt).strftime("%d/%m/%Y")
        except ValueError:
            pass
    # Remove ordinal suffixes: 1st, 2nd, 3rd, 4th, ...
    s2 = re.sub(r"(\d)(st|nd|rd|th)", r"\1", s, flags=re.IGNORECASE)
    # Common long-form formats
    for fmt in ("%B %d, %Y", "%b %d, %Y", "%B %d %Y", "%b %d %Y"):
        try:
            return datetime.strptime(s2, fmt).strftime("%d/%m/%Y")
        except ValueError:
            pass
    return None


def _coerce_int(value: Any) -> Optional[int]:
    if value is None:
        return None
    if isinstance(value, bool):
        return int(value)
    if isinstance(value, (int,)):
        return value
    if isinstance(value, float):
        return int(value)
    if isinstance(value, str):
        s = value.strip()
        if not s:
            return None
        try:
            # allow "0", "2", "2.0"
            return int(float(s))
        except ValueError:
            return None
    return None


async def sanitize_kiwi_flight_tool_args(request: MCPToolCallRequest, handler):
    # Only touch the flight search tool; let others pass through unchanged.
    if request.name != "search-flight":
        return await handler(request)

    args = dict(request.args or {})

    # Coerce flex ranges into numbers (the MCP tool schema expects numbers).
    for key in ("departureDateFlexRange", "returnDateFlexRange"):
        if key in args:
            coerced = _coerce_int(args.get(key))
            if coerced is None:
                # If blank/invalid, default to 0 rather than sending a string.
                args[key] = 0
            else:
                args[key] = coerced

    # Normalize dates to dd/mm/yyyy; omit returnDate if empty/invalid (one-way queries).
    if "departureDate" in args:
        normalized = _normalize_date_to_ddmmyyyy(args.get("departureDate"))
        if normalized is not None:
            args["departureDate"] = normalized

    if "returnDate" in args:
        normalized = _normalize_date_to_ddmmyyyy(args.get("returnDate"))
        if normalized is None:
            # Don’t send bad/empty returnDate; Kiwi MCP rejects it.
            args.pop("returnDate", None)
            args.pop("returnDateFlexRange", None)
        else:
            args["returnDate"] = normalized

    return await handler(request.override(args=args))

In [13]:
client = MultiServerMCPClient(
    {
        "kiwi-com-flight-search": {
            "transport": "streamable_http",
            "url": "https://mcp.kiwi.com",
        }
    },
    tool_interceptors=[sanitize_kiwi_flight_tool_args],
)

flight_search_tool = await client.get_tools()

model = ChatNVIDIA(model = "qwen/qwen3-next-80b-a3b-instruct", temperature=0.2)

flight_agent = create_agent(
    model=model,
    tools=flight_search_tool,
    #checkpointer=InMemorySaver(),
    system_prompt=("You are a helpful assistant that can search for flights. "
    "When calling tools: use dates in dd/mm/yyyy; use numbers (not strings) for flex ranges; "
    "and omit returnDate/returnDateFlexRange for one-way searches."),
)


In [14]:
from typing import Dict, Any
from tavily import TavilyClient

tavily_client = TavilyClient()

@tool
def web_search(query: str) -> Dict[str, Any]:

    """Search the web for information"""

    return tavily_client.search(query)

web_search_agent = create_agent(
    model=model,
    tools=[web_search],
    system_prompt="You are a helpful assistant who can search the web for information using the web search tool.",
)

In [ ]:
from langchain.messages import SystemMessage, HumanMessage

@tool
async def call_flight_agent(query: str) -> str:
    """ Call the flight agent with the given query and return its response as a string. The query will be a natural language question about flights, e.g. "Find me a flight from New York to London on December 15th". The flight agent will use the Kiwi flight search tool to find the answer, and you should return that answer as a string.
    """
    response = await flight_agent.ainvoke({"messages": [HumanMessage(content=query)]})
    return response["messages"][-1].content

@tool
async def call_web_search_agent(query: str) -> str:
    """ Call the web search agent with the given query and return its response as a string. The query will be a natural language question that may require searching the web for information, e.g. "What is the capital of France?" or "What are some good restaurants in New York?". The web search agent will use the web search tool to find the answer, and you should return that answer as a string.
    """
    response = await web_search_agent.ainvoke({"messages": [HumanMessage(content=query)]})
    return response["messages"][-1].content


main_agent = create_agent(
    model=model,
    tools=[call_flight_agent, call_web_search_agent],
    system_prompt=("You are a helpful assistant who plans weddings. You can search for flights, venues, suitable music, and general information about wedding-related topics."),
    checkpointer=InMemorySaver()
)


In [ ]:
# test

query = HumanMessage(content="I want to have my wedding in Paris in June 2026. Can you find me some flights from New York to Paris around that time, and also suggest some good venues in Paris for a wedding?")

response = await main_agent.ainvoke({"messages": [query]}, config={"configurable": {"thread_id": "my_wedding_planner_thread"}})

print(response["messages"][-1].content)

Here's a summary of your wedding planning options for Paris in June 2026:

### ✈️ **Flights from New York to Paris (June 2026)**

#### 💰 **Best Value Flights (Under €250)**
- **Newark (EWR) → Paris Orly (ORY)**  
  - Departure: June 2–3, 2026 at 11:00 PM → Arrival: 12:15 PM next day  
  - Duration: 7h 15m  
  - Price: **€200** (economy)  
  - [Book here](https://on.kiwi.com/KzkziF)

- **Newark (EWR) → Paris Charles de Gaulle (CDG)**  
  - Departure: June 1, 2026 at 5:00 PM → Arrival: June 2 at 9:05 PM  
  - Duration: 22h 25m (long layover)  
  - Price: **€272** (economy)  
  - [Book here](https://on.kiwi.com/HbVVrO)

#### ⏱️ **Fastest Direct Flights (Under 7h 30m)**
- **JFK → CDG**  
  - Departure: June 1, 2026 at 1:00 AM → Arrival: 2:10 PM  
  - Duration: **7h 10m**  
  - Price: **€449** (economy)  
  - [Book here](https://on.kiwi.com/FuuFGr)

> 💡 *Tip:* For the best balance of price and convenience, consider the **EWR → ORY** flights at €200 — they’re direct, affordable, and land at 